In [6]:
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import cv2 
import numpy as np 
import math 

os.environ["CUDA_VISIBLE_DEVICES"] = '1'
print(tf.__version__)

AttributeError: module 'tensorflow' has no attribute '__version__'

### Define number of train and test images

In [0]:
!nvidia-smi

## Generate and save Train Data

In [0]:
path = '/content/drive/My Drive/Drive2(Panasonic)/Dataset/CelebA/'
classes = os.listdir(path)
train_size = 704
file_size = 128
file_size2 = 32
print(classes)
index = 0
counter = 0
X_train = np.zeros((train_size, int(file_size2), int(file_size2), 3), dtype = 'uint8')
Y_image = np.zeros((train_size, int(file_size), int(file_size), 3), dtype = 'uint8')
#Y_image = original image
#X_train = downscaled image
Y_train = np.zeros((train_size), dtype = 'uint8')

for i in classes:

    print('class: ', i)
    files = os.listdir(str(path) + str(i))

    for k in files:
        img = Image.open(str(path) + str(i) + '/' + str(k))
        img.load
        img = img.resize((int(file_size), int(file_size)), Image.ANTIALIAS)
        img_down = img.resize((int(file_size2), int(file_size2)), Image.ANTIALIAS)
        npimg = np.asarray( img, dtype="uint8" )
        npimg_down = np.asarray( img_down, dtype="uint8" )
        X_train[counter,:,:,:] = npimg_down
        Y_image[counter,:,:,:] = npimg
        Y_train[counter] = classes.index(i)
        counter += 1
    
      
print(counter)

np.save('/content/drive/My Drive/Drive1(Own)/Arrow224/X_train.npy', X_train)
np.save('/content/drive/My Drive/Drive1(Own)/Arrow224/Y_train.npy', Y_train)
np.save('/content/drive/My Drive/Drive1(Own)/Arrow224/Y_image.npy', Y_image)
print(X_train.shape, Y_image.shape, Y_train.shape)


['right', 'left', 'none']
class:  right
class:  left
class:  none
704
(704, 32, 32, 3) (704, 128, 128, 3) (704,)


## Load and Normalize Train and Test Data

In [8]:
path_load = '/media/dl/DL/aashish/upscaling/sr_dataset/'
X_train = np.load(path_load + 'X_train.npy')
Y_image = np.load(path_load + 'Y_image.npy')
Y_train = np.load(path_load + 'Y_train.npy')
num_classes = 1
'''X_train = np.load('/content/drive/My Drive/Drive1(Own)/Arrow224/X_train.npy')
Y_image = np.load('/content/drive/My Drive/Drive1(Own)/Arrow224/Y_image.npy')
Y_train = np.load('/content/drive/My Drive/Drive1(Own)/Arrow224/Y_train.npy')'''


X_train = (X_train[:,:,:,:]/255).astype('float32')
Y_image = (Y_image[:,:,:,:]/255).astype('float32')

#X_celeb = X_celeb.astype('float16')
#X_celeb = np.resize(X_celeb, (train_size, 160, 160, 3))

#X_test = testx/255
#X_test = X_test.astype('float16')
#X_test = np.resize(X_test, (test_size, 64, 64, 3))

def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y
  
Y_train = convert_to_one_hot(Y_train[:], num_classes).T  
#Y_test = convert_to_one_hot(testy, 8).T

print(X_train.shape, Y_image.shape, Y_train.shape)
#print(X_test.shape, Y_test.shape)

(20000, 32, 32, 3) (20000, 256, 256, 3) (20000, 1)


## Define Training Network

In [9]:
def srcnn(X):

    gen1 = tf.layers.conv2d(X, 64, 9, 1, padding = 'same')
    P1 = tf.nn.relu(gen1)    
    
    gen2 = tf.layers.conv2d(P1, 32, 1, 1, padding = 'SAME')
    P2 = tf.nn.relu(gen2)    

    res_in_1 = P2
    rc1 = tf.layers.conv2d(res_in_1, 32, 3, 1, padding = 'SAME')
    rc1 = tf.nn.relu(rc1)
    r1 = tf.layers.conv2d(rc1, 32, 1, 1, padding = 'SAME')
    r1 = tf.nn.relu(r1)
    r1 = tf.layers.conv2d(r1, 32, 1, 1, padding = 'SAME')
    r1 = tf.nn.relu(r1)
    res_out_1 = res_in_1 + r1    

    res_in_2 = res_out_1
    rc2 = tf.layers.conv2d(res_in_2, 32, 3, 1, padding = 'SAME')
    rc2 = tf.nn.relu(rc2)
    r2 = tf.layers.conv2d(rc2, 32, 1, 1, padding = 'SAME')
    r2 = tf.nn.relu(r2)
    r2 = tf.layers.conv2d(r2, 32, 1, 1, padding = 'SAME')
    r2 = tf.nn.relu(r2)
    res_out_2 = res_in_2 + r2 

    res_in_3 = res_out_2
    rc3 = tf.layers.conv2d(res_in_3, 32, 3, 1, padding = 'SAME')
    rc3 = tf.nn.relu(rc3)
    r3 = tf.layers.conv2d(rc3, 32, 1, 1, padding = 'SAME')
    r3 = tf.nn.relu(r3)
    r3 = tf.layers.conv2d(r3, 32, 1, 1, padding = 'SAME')
    r3 = tf.nn.relu(r3)
    res_out_3 = res_in_3 + r3

    res_in_4 = res_out_3
    rc4 = tf.layers.conv2d(res_in_4, 32, 3, 1, padding = 'SAME')
    rc4 = tf.nn.relu(rc4)
    r4 = tf.layers.conv2d(rc4, 32, 1, 1, padding = 'SAME')
    r4 = tf.nn.relu(r4)
    r4 = tf.layers.conv2d(r4, 32, 1, 1, padding = 'SAME')
    r4 = tf.nn.relu(r4)
    res_out_4 = res_in_4 + r4    

    res_in_5 = res_out_4
    rc5 = tf.layers.conv2d(res_in_5, 32, 3, 1, padding = 'SAME')
    rc5 = tf.nn.relu(rc5)
    r5 = tf.layers.conv2d(rc5, 32, 1, 1, padding = 'SAME')
    r5 = tf.nn.relu(r5)
    r5 = tf.layers.conv2d(r5, 32, 1, 1, padding = 'SAME')
    r5 = tf.nn.relu(r5)
    res_out_5 = res_in_5 + r5    

    res_in_5 = res_out_5
    rc5 = tf.layers.conv2d(res_in_5, 32, 3, 1, padding = 'SAME')
    rc5 = tf.nn.relu(rc5)
    r5 = tf.layers.conv2d(rc5, 32, 1, 1, padding = 'SAME')
    r5 = tf.nn.relu(r5)
    r5 = tf.layers.conv2d(r5, 32, 1, 1, padding = 'SAME')
    r5 = tf.nn.relu(r5)
    res_out_5 = res_in_5 + r5 

    regen1 = tf.layers.conv2d(P2, 64, 5, 1, padding = 'same')

    #print('res_out shape:', res_out_2.shape)
    upsam = Upsample2xBlock(P2, kernel_size=3, filters=64)#tf.layers.conv2d_transpose(res_out_3,64,(3, 3),strides=(4,4), padding='same')
    #[batch, height, width, in_channels]
    #[height, width, output_channels, in_channels]
    print('upsam shape:', upsam.shape)

    res_in_5 = upsam
    rc5 = tf.layers.conv2d(res_in_5, 32, 3, 1, padding = 'SAME')
    rc5 = tf.nn.relu(rc5)
    r5 = tf.layers.conv2d(rc5, 32, 1, 1, padding = 'SAME')
    r5 = tf.nn.relu(r5)
    r5 = tf.layers.conv2d(r5, 4, 1, 1, padding = 'SAME')
    r5 = tf.nn.relu(r5)
    res_out_5 = res_in_5 + r5 

    res_in_5 = res_out_5
    rc5 = tf.layers.conv2d(res_in_5, 32, 3, 1, padding = 'SAME')
    rc5 = tf.nn.relu(rc5)
    r5 = tf.layers.conv2d(rc5, 32, 1, 1, padding = 'SAME')
    r5 = tf.nn.relu(r5)
    r5 = tf.layers.conv2d(r5, 4, 1, 1, padding = 'SAME')
    r5 = tf.nn.relu(r5)
    res_out_5 = res_in_5 + r5 

    res_in_5 = res_out_5
    rc5 = tf.layers.conv2d(res_in_5, 32, 3, 1, padding = 'SAME')
    rc5 = tf.nn.relu(rc5)
    r5 = tf.layers.conv2d(rc5, 32, 1, 1, padding = 'SAME')
    r5 = tf.nn.relu(r5)
    r5 = tf.layers.conv2d(r5, 4, 1, 1, padding = 'SAME')
    r5 = tf.nn.relu(r5)
    res_out_5 = res_in_5 + r5 

    res_in_5 = res_out_5
    rc5 = tf.layers.conv2d(res_in_5, 32, 3, 1, padding = 'SAME')
    rc5 = tf.nn.relu(rc5)
    r5 = tf.layers.conv2d(rc5, 32, 1, 1, padding = 'SAME')
    r5 = tf.nn.relu(r5)
    r5 = tf.layers.conv2d(r5, 4, 1, 1, padding = 'SAME')
    r5 = tf.nn.relu(r5)
    res_out_5 = res_in_5 + r5 

    res_in_5 = res_out_5
    rc5 = tf.layers.conv2d(res_in_5, 32, 3, 1, padding = 'SAME')
    rc5 = tf.nn.relu(rc5)
    r5 = tf.layers.conv2d(rc5, 32, 1, 1, padding = 'SAME')
    r5 = tf.nn.relu(r5)
    r5 = tf.layers.conv2d(r5, 4, 1, 1, padding = 'SAME')
    r5 = tf.nn.relu(r5)
    res_out_5 = res_in_5 + r5 

    regen1 = tf.layers.conv2d(upsam, 3, 3, 1, padding = 'same')
    A3 = tf.nn.relu(regen1)     
    regen2 = tf.layers.conv2d(A3, 3, 3, 1, padding = 'same')
    
    print('output shape: ', regen2.shape)
    
    return regen1

def inception(X):

    A6, _ = inception_resnet_v2(X)

    P_fl = tf.contrib.layers.flatten(A6)
    fc = tf.contrib.layers.fully_connected(P_fl, num_classes, activation_fn = None)
    
    return fc


def inference(input_images):
    with slim.arg_scope([slim.conv2d], 
                         activation_fn=tf.nn.relu, stride=1, padding='SAME',
                         weights_initializer=tf.truncated_normal_initializer(stddev=0.01)):
                         # weights_initializer=tf.contrib.layers.xavier_initializer()):
        x = slim.conv2d(input_images, 32, [3, 3],
                        weights_initializer=tf.contrib.layers.xavier_initializer(),
                        padding='VALID', scope='conv1a')

        x = slim.conv2d(x, 64, [3, 3], 
                        weights_initializer=tf.contrib.layers.xavier_initializer(), 
                        padding='VALID', scope='conv1b')

        pool1b = slim.max_pool2d(x, [2, 2], stride=2, padding='VALID', scope='pool1b')

        conv2_1 = slim.conv2d(pool1b, 64, [3, 3], scope='conv2_1')
        conv2_2 = slim.conv2d(conv2_1, 64, [3, 3], scope='conv2_2')
        res2_2 = pool1b + conv2_2
        conv2 = slim.conv2d(res2_2, 128, [3, 3],
                        weights_initializer=tf.contrib.layers.xavier_initializer(),
                        padding='VALID', scope='conv2')

        pool2 = slim.max_pool2d(conv2, [2, 2], stride=2, padding='VALID', scope='pool2')
        conv3_1 = slim.conv2d(pool2, 128, [3, 3], scope='conv3_1')
        conv3_2 = slim.conv2d(conv3_1, 128, [3, 3], scope='conv3_2')
        res3_2 = pool2 + conv3_2

        conv3_3 = slim.conv2d(res3_2, 128, [3, 3], scope='conv3_3')
        conv3_4 = slim.conv2d(conv3_3, 128, [3, 3], scope='conv3_4')
        res3_4 = res3_2 + conv3_4

        conv3 = slim.conv2d(res3_4, 256, [3, 3],
                        weights_initializer=tf.contrib.layers.xavier_initializer(),
                        padding='VALID', scope='conv3')
        pool3 = slim.max_pool2d(conv3, [2, 2], stride=2, padding='VALID', scope='pool3')
        conv4_1 = slim.conv2d(pool3, 256, [3, 3], scope='conv4_1')
        conv4_2 = slim.conv2d(conv4_1, 256, [3, 3], scope='conv4_2')
        res4_2 = pool3 + conv4_2

        conv4_3 = slim.conv2d(res4_2, 256, [3, 3], scope='conv4_3')
        conv4_4 = slim.conv2d(conv4_3, 256, [3, 3], scope='conv4_4')
        res4_4 = res4_2 + conv4_4

        conv4_5 = slim.conv2d(res4_4, 256, [3, 3], scope='conv4_5')
        conv4_6 = slim.conv2d(conv4_5, 256, [3, 3], scope='conv4_6')
        res4_6 = res4_4 + conv4_6

        conv4_7 = slim.conv2d(res4_6, 256, [3, 3], scope='conv4_7')
        conv4_8 = slim.conv2d(conv4_7, 256, [3, 3], scope='conv4_8')
        res4_8 = res4_6 + conv4_8

        conv4_9 = slim.conv2d(res4_8, 256, [3, 3], scope='conv4_9')
        conv4_10 = slim.conv2d(conv4_9, 256, [3, 3], scope='conv4_10')
        res4_10 = res4_8 + conv4_10

        conv4 = slim.conv2d(res4_10, 512, [3, 3],
                        weights_initializer=tf.contrib.layers.xavier_initializer(),
                        padding='VALID', scope='conv4')
        pool4 = slim.max_pool2d(conv4, [2, 2], stride=2, padding='VALID', scope='pool4')
        
        conv5_1 = slim.conv2d(pool4, 512, [3, 3], scope='conv5_1')
        conv5_2 = slim.conv2d(conv5_1, 512, [3, 3], scope='conv5_2')
        res5_2 = pool4 + conv5_2

        conv5_3 = slim.conv2d(res5_2, 512, [3, 3], scope='conv5_3')
        conv5_4 = slim.conv2d(conv5_3, 512, [3, 3], scope='conv5_4')
        res5_4 = res5_2 + conv5_4

        conv5_5 = slim.conv2d(res5_4, 512, [3, 3], scope='conv5_5')
        conv5_6 = slim.conv2d(conv5_5, 512, [3, 3], scope='conv5_6')
        res5_6 = res5_4 + conv5_6
        res5_6 = slim.flatten(res5_6, scope='flatten')
        feature = slim.fully_connected(res5_6, num_outputs=512, activation_fn=None, 
                            weights_initializer=tf.contrib.layers.xavier_initializer(), scope='fc1')

        x = slim.fully_connected(feature, num_outputs=num_classes, activation_fn=None, scope='fc2')
    
    return x




In [0]:
# Inception-Renset-A
def block35(net, scale=1.0, activation_fn=tf.nn.relu, scope=None, reuse=None):
    """Builds the 35x35 resnet block."""
    with tf.variable_scope(scope, 'Block35', [net], reuse=reuse):
        with tf.variable_scope('Branch_0'):
            tower_conv = slim.conv2d(net, 32, 1, scope='Conv2d_1x1')
        with tf.variable_scope('Branch_1'):
            tower_conv1_0 = slim.conv2d(net, 32, 1, scope='Conv2d_0a_1x1')
            tower_conv1_1 = slim.conv2d(tower_conv1_0, 32, 3, scope='Conv2d_0b_3x3')
        with tf.variable_scope('Branch_2'):
            tower_conv2_0 = slim.conv2d(net, 32, 1, scope='Conv2d_0a_1x1')
            tower_conv2_1 = slim.conv2d(tower_conv2_0, 48, 3, scope='Conv2d_0b_3x3')
            tower_conv2_2 = slim.conv2d(tower_conv2_1, 64, 3, scope='Conv2d_0c_3x3')
        mixed = tf.concat([tower_conv, tower_conv1_1, tower_conv2_2], 3)
        up = slim.conv2d(mixed, net.get_shape()[3], 1, normalizer_fn=None,
                         activation_fn=None, scope='Conv2d_1x1')
        net += scale * up
        if activation_fn:
            net = activation_fn(net)
    return net

# Inception-Renset-B
def block17(net, scale=1.0, activation_fn=tf.nn.relu, scope=None, reuse=None):
    """Builds the 17x17 resnet block."""
    with tf.variable_scope(scope, 'Block17', [net], reuse=reuse):
        with tf.variable_scope('Branch_0'):
            tower_conv = slim.conv2d(net, 192, 1, scope='Conv2d_1x1')
        with tf.variable_scope('Branch_1'):
            tower_conv1_0 = slim.conv2d(net, 128, 1, scope='Conv2d_0a_1x1')
            tower_conv1_1 = slim.conv2d(tower_conv1_0, 160, [1, 7],
                                        scope='Conv2d_0b_1x7')
            tower_conv1_2 = slim.conv2d(tower_conv1_1, 192, [7, 1],
                                        scope='Conv2d_0c_7x1')
        mixed = tf.concat([tower_conv, tower_conv1_2], 3)
        up = slim.conv2d(mixed, net.get_shape()[3], 1, normalizer_fn=None,
                         activation_fn=None, scope='Conv2d_1x1')
        net += scale * up
        if activation_fn:
            net = activation_fn(net)
    return net


# Inception-Resnet-C
def block8(net, scale=1.0, activation_fn=tf.nn.relu, scope=None, reuse=None):
    """Builds the 8x8 resnet block."""
    with tf.variable_scope(scope, 'Block8', [net], reuse=reuse):
        with tf.variable_scope('Branch_0'):
            tower_conv = slim.conv2d(net, 192, 1, scope='Conv2d_1x1')
        with tf.variable_scope('Branch_1'):
            tower_conv1_0 = slim.conv2d(net, 192, 1, scope='Conv2d_0a_1x1')
            tower_conv1_1 = slim.conv2d(tower_conv1_0, 224, [1, 3],
                                        scope='Conv2d_0b_1x3')
            tower_conv1_2 = slim.conv2d(tower_conv1_1, 256, [3, 1],
                                        scope='Conv2d_0c_3x1')
        mixed = tf.concat([tower_conv, tower_conv1_2], 3)
        up = slim.conv2d(mixed, net.get_shape()[3], 1, normalizer_fn=None,
                         activation_fn=None, scope='Conv2d_1x1')
        net += scale * up
        if activation_fn:
            net = activation_fn(net)
    return net
  
def inference(images, keep_probability, phase_train=True, 
              bottleneck_layer_size=128, weight_decay=0.0, reuse=None):
    batch_norm_params = {
        # Decay for the moving averages.
        'decay': 0.995,
        # epsilon to prevent 0s in variance.
        'epsilon': 0.001,
        # force in-place updates of mean and variance estimates
        'updates_collections': None,
        # Moving averages ends up in the trainable variables collection
        'variables_collections': [ tf.GraphKeys.TRAINABLE_VARIABLES ],
}
    with slim.arg_scope([slim.conv2d, slim.fully_connected],
                        weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
                        weights_regularizer=slim.l2_regularizer(weight_decay),
                        normalizer_fn=slim.batch_norm,
                        normalizer_params=batch_norm_params):
        return inception_resnet_v2(images, is_training=phase_train,
              dropout_keep_prob=keep_probability, bottleneck_layer_size=bottleneck_layer_size, reuse=reuse)


def inception_resnet_v2(inputs, is_training=True,
                        dropout_keep_prob=0.8,
                        bottleneck_layer_size=128,
                        reuse=None,
                        scope='InceptionResnetV2'):
    """Creates the Inception Resnet V2 model.
    Args:
      inputs: a 4-D tensor of size [batch_size, height, width, 3].
      num_classes: number of predicted classes.
      is_training: whether is training or not.
      dropout_keep_prob: float, the fraction to keep before final layer.
      reuse: whether or not the network and its variables should be reused. To be
        able to reuse 'scope' must be given.
      scope: Optional variable_scope.
    Returns:
      logits: the logits outputs of the model.
      end_points: the set of end_points from the inception model.
    """
    end_points = {}
  
    with tf.variable_scope(scope, 'InceptionResnetV2', [inputs], reuse=reuse):
        with slim.arg_scope([slim.batch_norm, slim.dropout],
                            is_training=is_training):
            with slim.arg_scope([slim.conv2d, slim.max_pool2d, slim.avg_pool2d],
                                stride=1, padding='SAME'):
      
                # 149 x 149 x 32
                net = slim.conv2d(inputs, 32, 3, stride=2, padding='VALID',
                                  scope='Conv2d_1a_3x3')
                end_points['Conv2d_1a_3x3'] = net
                # 147 x 147 x 32
                net = slim.conv2d(net, 32, 3, padding='VALID',
                                  scope='Conv2d_2a_3x3')
                end_points['Conv2d_2a_3x3'] = net
                # 147 x 147 x 64
                net = slim.conv2d(net, 64, 3, scope='Conv2d_2b_3x3')
                end_points['Conv2d_2b_3x3'] = net
                # 73 x 73 x 64
                net = slim.max_pool2d(net, 3, stride=2, padding='VALID',
                                      scope='MaxPool_3a_3x3')
                end_points['MaxPool_3a_3x3'] = net
                # 73 x 73 x 80
                net = slim.conv2d(net, 80, 1, padding='VALID',
                                  scope='Conv2d_3b_1x1')
                end_points['Conv2d_3b_1x1'] = net
                # 71 x 71 x 192
                net = slim.conv2d(net, 192, 3, padding='VALID',
                                  scope='Conv2d_4a_3x3')
                end_points['Conv2d_4a_3x3'] = net
                # 35 x 35 x 192
                net = slim.max_pool2d(net, 3, stride=2, padding='VALID',
                                      scope='MaxPool_5a_3x3')
                end_points['MaxPool_5a_3x3'] = net
        
                # 35 x 35 x 320
                with tf.variable_scope('Mixed_5b'):
                    with tf.variable_scope('Branch_0'):
                        tower_conv = slim.conv2d(net, 96, 1, scope='Conv2d_1x1')
                    with tf.variable_scope('Branch_1'):
                        tower_conv1_0 = slim.conv2d(net, 48, 1, scope='Conv2d_0a_1x1')
                        tower_conv1_1 = slim.conv2d(tower_conv1_0, 64, 5,
                                                    scope='Conv2d_0b_5x5')
                    with tf.variable_scope('Branch_2'):
                        tower_conv2_0 = slim.conv2d(net, 64, 1, scope='Conv2d_0a_1x1')
                        tower_conv2_1 = slim.conv2d(tower_conv2_0, 96, 3,
                                                    scope='Conv2d_0b_3x3')
                        tower_conv2_2 = slim.conv2d(tower_conv2_1, 96, 3,
                                                    scope='Conv2d_0c_3x3')
                    with tf.variable_scope('Branch_3'):
                        tower_pool = slim.avg_pool2d(net, 3, stride=1, padding='SAME',
                                                     scope='AvgPool_0a_3x3')
                        tower_pool_1 = slim.conv2d(tower_pool, 64, 1,
                                                   scope='Conv2d_0b_1x1')
                    net = tf.concat([tower_conv, tower_conv1_1,
                                        tower_conv2_2, tower_pool_1], 3)
        
                end_points['Mixed_5b'] = net
                net = slim.repeat(net, 10, block35, scale=0.17)
        
                # 17 x 17 x 1024
                with tf.variable_scope('Mixed_6a'):
                    with tf.variable_scope('Branch_0'):
                        tower_conv = slim.conv2d(net, 384, 3, stride=2, padding='VALID',
                                                 scope='Conv2d_1a_3x3')
                    with tf.variable_scope('Branch_1'):
                        tower_conv1_0 = slim.conv2d(net, 256, 1, scope='Conv2d_0a_1x1')
                        tower_conv1_1 = slim.conv2d(tower_conv1_0, 256, 3,
                                                    scope='Conv2d_0b_3x3')
                        tower_conv1_2 = slim.conv2d(tower_conv1_1, 384, 3,
                                                    stride=2, padding='VALID',
                                                    scope='Conv2d_1a_3x3')
                    with tf.variable_scope('Branch_2'):
                        tower_pool = slim.max_pool2d(net, 3, stride=2, padding='VALID',
                                                     scope='MaxPool_1a_3x3')
                    net = tf.concat([tower_conv, tower_conv1_2, tower_pool], 3)
        
                end_points['Mixed_6a'] = net
                net = slim.repeat(net, 20, block17, scale=0.10)
        
                with tf.variable_scope('Mixed_7a'):
                    with tf.variable_scope('Branch_0'):
                        tower_conv = slim.conv2d(net, 256, 1, scope='Conv2d_0a_1x1')
                        tower_conv_1 = slim.conv2d(tower_conv, 384, 3, stride=2,
                                                   padding='VALID', scope='Conv2d_1a_3x3')
                    with tf.variable_scope('Branch_1'):
                        tower_conv1 = slim.conv2d(net, 256, 1, scope='Conv2d_0a_1x1')
                        tower_conv1_1 = slim.conv2d(tower_conv1, 288, 3, stride=2,
                                                    padding='VALID', scope='Conv2d_1a_3x3')
                    with tf.variable_scope('Branch_2'):
                        tower_conv2 = slim.conv2d(net, 256, 1, scope='Conv2d_0a_1x1')
                        tower_conv2_1 = slim.conv2d(tower_conv2, 288, 3,
                                                    scope='Conv2d_0b_3x3')
                        tower_conv2_2 = slim.conv2d(tower_conv2_1, 320, 3, stride=2,
                                                    padding='VALID', scope='Conv2d_1a_3x3')
                    with tf.variable_scope('Branch_3'):
                        tower_pool = slim.max_pool2d(net, 3, stride=2, padding='VALID',
                                                     scope='MaxPool_1a_3x3')
                    net = tf.concat([tower_conv_1, tower_conv1_1,
                                        tower_conv2_2, tower_pool], 3)
        
                end_points['Mixed_7a'] = net
        
                net = slim.repeat(net, 9, block8, scale=0.20)
                net = block8(net, activation_fn=None)
        
                net = slim.conv2d(net, 1536, 1, scope='Conv2d_7b_1x1')
                end_points['Conv2d_7b_1x1'] = net
        
                with tf.variable_scope('Logits'):
                    end_points['PrePool'] = net
                    #pylint: disable=no-member
                    net = slim.avg_pool2d(net, net.get_shape()[1:3], padding='VALID',
                                          scope='AvgPool_1a_8x8')
                    net = slim.flatten(net)
          
                    net = slim.dropout(net, dropout_keep_prob, is_training=is_training,
                                       scope='Dropout')
          
                    end_points['PreLogitsFlatten'] = net
                
                net = slim.fully_connected(net, bottleneck_layer_size, activation_fn=None, 
                        scope='Bottleneck', reuse=False)
  
    return net, end_points


In [11]:
def random_mini_batches(X, Y_img, Y, mini_batch_size = 64, seed = 10):
    
    m = X.shape[0]                  # number of training examples
    mini_batches = []
    np.random.seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation,:,:,:]
    shuffled_Y_img = Y_img[permutation,:,:,:]
    shuffled_Y = Y[permutation,:]

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:,:,:]
        mini_batch_Y_img = shuffled_Y_img[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:,:,:]
        mini_batch_Y = shuffled_Y[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:]
        mini_batch = (mini_batch_X, mini_batch_Y_img, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[num_complete_minibatches * mini_batch_size : m,:,:,:]
        mini_batch_Y_img = shuffled_Y_img[num_complete_minibatches * mini_batch_size : m,:,:,:]
        mini_batch_Y = shuffled_Y[num_complete_minibatches * mini_batch_size : m,:]
        mini_batch = (mini_batch_X, mini_batch_Y_img, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

def PSNR(gt, pred):
    gt = gt.astype(np.float64)
    pred = pred.astype(np.float64)
    mse = np.mean((pred - gt)**2)
    psnr = 10*np.log10(255*255/mse)
    return psnr
    #return compare_psnr(gt, pred, data_range=255)
    
def SSIM(gt, pred):
    ssim = compare_ssim(gt, pred, data_range=255, gaussian_weights=True)
    return ssim

def Upsample2xBlock(x, kernel_size, filters, strides=1):
    x = tf.layers.conv2d(x, kernel_size=kernel_size, filters=filters, strides=strides, padding='same')
    x = tf.depth_to_space(x, 8)
    x = tf.nn.relu(x)
    return x

## Training Model


In [17]:
minibatch_size = 128
learning_rate = 0.005
num_epochs = 100001
seed = 10
print('X_train shape', X_train.shape)  
print('Y_train shape', Y_train.shape)  
print('Learning rate:', learning_rate)

(m, n_H0, n_W0, n_C0) = X_train.shape  
(m1, n_H1, n_W1, n_C1) = Y_image.shape             
n_y = Y_train.shape[1]                            
costs = []                                                            
t1 = 0
t2 = 0

X = tf.placeholder(tf.float32, [None, n_H0, n_W0, n_C0], name = 'X')
Y_img = tf.placeholder(tf.float32, [None, n_H1, n_W1, n_C1], name = 'Y_img')
Y = tf.placeholder(tf.float32, [None, n_y], name = 'Y')

Z3 = srcnn(X)
#Z4 = squeeze_net(input=Z3, classes=num_classes)
#Z4 = inception(Z3)
#Z4 = inference(Z3)

cost1 = tf.losses.mean_squared_error(Z3, Y_img)
cost1 = tf.reduce_mean(cost1)
#cost2 = tf.nn.softmax_cross_entropy_with_logits(logits = Z4, labels = Y)
#cost2 = tf.reduce_mean(cost2)
cost = cost1 #+ cost2

#psnr = cost1 #PSNR(Y_img, Z3)
#ssim = cost2 #SSIM(Y_img, Z3)
#psnr = 10*np.log10(255*255/cost1)

optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)    
init = tf.global_variables_initializer()    
saver = tf.train.Saver()

with tf.Session() as sess:

    sess.run(init)

    print('Trainable Params:', np.sum([np.prod(v.shape) for v in tf.trainable_variables()]))

    for epoch in range(num_epochs):
        
        minibatch_cost = 0.
        num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
        seed = seed + 1
        minibatches = random_mini_batches(X_train, Y_image, Y_train, minibatch_size, seed)
        for minibatch in minibatches:

            (minibatch_X, minibatch_Yi, minibatch_Y) = minibatch
            _ , temp_cost, temp_cost1 = sess.run([optimizer, cost, cost1], feed_dict = {X: minibatch_X, Y_img: minibatch_Yi})

            #temp_cost += temp_cost / num_minibatches
            temp_cost1 += temp_cost1 / num_minibatches

        if epoch % 10 == 0:
            psnr = 10*np.log10(np.max(minibatch_Yi)*np.max(minibatch_Yi)/temp_cost1)
            costs.append(temp_cost)
            t2 = time.time()
            print ("Epoch:", epoch, 'Time:', round(t2-t1, 1), 'Total loss:', round(temp_cost, 6), 'SR loss:', round(temp_cost1, 6), 'PSNR:', round(psnr, 6))
            t1 = time.time()

        if epoch % 50 == 0:
            print('Saving Model...')
            saver.save(sess, '/content/drive/My Drive/Drive2(Panasonic)/Dataset/CelebA/model/model.ckpt')
            print('Model Saved...')
            Z31 = Z3[0].eval(feed_dict = {X: minibatch_X, Y_img: minibatch_Yi})
            plt.imshow(Z31)
            plt.show()
            #!nvidia-smi


    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()

    predict_op = tf.argmax(Z4, 1)
    correct_prediction = tf.equal(predict_op, tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print('accuracy', accuracy)
    train_accuracy = accuracy.eval({X: X_train, Y: Y_train})
    #test_accuracy = accuracy.eval({X: X_test, Y: Y_test})
    print("Train Accuracy:", train_accuracy)
    #print("Test Accuracy:", test_accuracy)
                 

X_train shape (20000, 32, 32, 3)
Y_train shape (20000, 1)
Learning rate: 0.005


AttributeError: module 'tensorflow' has no attribute 'placeholder'

In [0]:

tf.reset_default_graph()
imported_meta = tf.train.import_meta_graph("/content/drive/My Drive/Drive2(Panasonic)/Dataset/CelebA/model/model.ckpt.meta")

learning_rate = 0.0001
num_epochs = 100
minibatch_size = 64
seed = 10
                           

costs = []                                                            
t1 = 0
t2 = 0

(m, n_H0, n_W0, n_C0) = X_train.shape  
#(m1, n_H1, n_W1, n_C1) = Y_image.shape             
n_y = Y_train.shape[1]                            
costs = []                                                            
t1 = 0
t2 = 0

X = tf.placeholder(tf.float32, [None, n_H0, n_W0, n_C0], name = 'X')
#Y_img = tf.placeholder(tf.float32, [None, n_H1, n_W1, n_C1], name = 'Y_img')
Y = tf.placeholder(tf.float32, [None, n_y], name = 'Y')

Z3 = srcnn(X)
#Z4 = inference(Z3)
Z4 = inception(Z3)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    imported_meta.restore(sess, '/content/drive/My Drive/Drive2(Panasonic)/Dataset/CelebA/model/model.ckpt')
    print("restored")
    #print(np.sum([np.prod(v.shape) for v in tf.trainable_variables()]))
    correct_prediction = tf.equal(tf.argmax(Z4, 1), tf.argmax(Y, 1))
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    train_accuracy = correct_prediction.eval({X: X_train[0:1000,:,:,:], Y: Y_train[0:1000]})
    #, X_t: X_tiny, Y_t:Y_tiny
    print("Train Accuracy:", train_accuracy)

    #5yy73k

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
initial = [1,2,3,4]
initial.append(2)
file=open('/content/cost.txt','w')
file.write(str(cost))
file.close()